In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import mojimoji
import re
import jaconv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import warnings
from sklearn import preprocessing
import MeCab
def parse_text(text, debug=False):
    text = re.sub(r'［[^］]+］', ' ', text)  
#     text = re.sub(r'（[^）]+）', ' ', text)  
    text = re.sub(r'○', ' ', text)    
    text = re.sub(r'×', '', text)
    text = re.sub(r'※', '', text)    
#     text = re.sub(r'｜', ' ', text)
    text = re.sub(r'[\s、]', '' , text)
#     text = re.sub(r'一', ' ', text)
#     text = re.sub(r'…', ' ', text)  
#     text = re.sub(r'―', ' ', text)
    text = re.sub(r'[0-9]', '0', text)
    return text

def tokenize1(text):
    available_norm = ['接尾', '一般', '形容動詞語幹', 'サ変接続']
    node = mecab.parseToNode(text)
    l = []
    while node:
        l.append(node.surface)
        node = node.next
    return l
traindf=pd.read_csv("data/train.csv")
testdf=pd.read_csv("data/test.csv")
df=pd.concat([traindf,testdf])
df.body=df.body.map(parse_text)
traindf.body=traindf.body.map(parse_text)
testdf.body=testdf.body.map(parse_text)
y = df['author'].values
mecab = MeCab.Tagger('-Owakati')
count = CountVectorizer(analyzer=tokenize1)
bags = count.fit_transform(df.body.values)

features = count.get_feature_names()

bodyvec = pd.DataFrame(bags.toarray(), columns=features)
wordbody=bodyvec.sum().sort_values(ascending=False)
bodyvec1 = bodyvec[list(wordbody[wordbody > 100].index)]
xsum = bodyvec1.sum(axis=1)

xsum = np.array(xsum).reshape(len(xsum), 1)
newdf = pd.concat([df.reset_index(drop=True), pd.DataFrame(bodyvec1/xsum*100)], axis=1)
train=newdf.dropna().drop(["writing_id","body",],axis=1)
test=newdf[newdf.author.isnull()].drop(["writing_id","body"],axis=1)
test=test.drop(["author"],axis=1)
X = train.drop(["author"],axis=1)
y = train.author

mmscaler = preprocessing.MinMaxScaler()

mmscaler.fit(pd.concat([X,test]))
X = mmscaler.transform(X)
test= mmscaler.transform(test)


In [5]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 0) # 80%のデータを学習データに、20%を検証データにする
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(train_X,train_y)
pred = np.where(mlpr.predict(test_X)>0.38,1,0)
print('confusion matrix = \n', confusion_matrix(y_true=test_y, y_pred=pred))
print('accuracy = ', accuracy_score(y_true=test_y, y_pred=pred))
print('precision = ', precision_score(y_true=test_y, y_pred=pred))
print('recall = ', recall_score(y_true=test_y, y_pred=pred))
print('f1 score = ', f1_score(y_true=test_y, y_pred=pred))

confusion matrix = 
 [[919   0]
 [  1  74]]
accuracy =  0.9989939637826962
precision =  1.0
recall =  0.9866666666666667
f1 score =  0.9932885906040269


In [7]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
mlpr=MLPRegressor(**{"hidden_layer_sizes":(128,128,128),"random_state":42})
mlpr.fit(X,y)
pred = np.where(mlpr.predict(test)>0.3,1,0)
sub = pd.DataFrame(pd.read_csv("data/test.csv")['writing_id'])
sub["author"] = list(pred)
sub.to_csv("submission.csv", index = False)

In [8]:
data=pd.concat([pd.concat([pd.DataFrame(X),pd.DataFrame(test)]),pd.concat([y,pd.Series(pred)])],axis=1)
data.columns=list(wordbody[wordbody>100].index)+["target"]
data

,の,。,に,は,た,て,を,が,で,と,...,みつ子,曲る,説得,野良犬,血の気,鍛錬,ごころ,飛ば,颱風,target
0,0.455741,0.323014,0.579710,0.391644,0.171900,0.200348,0.521739,0.219130,0.469565,0.104348,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.397998,0.205155,0.515464,0.487536,0.259844,0.237526,0.386598,0.375773,0.306186,0.293814,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.366221,0.230725,0.217391,0.342688,0.515700,0.667826,0.478261,0.195652,0.391304,0.173913,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.370006,0.351721,0.445736,0.445153,0.606848,0.285767,0.562791,0.288837,0.117209,0.181395,...,0.0,0.204264,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0
4,0.455923,0.412082,0.528864,0.493165,0.693555,0.416179,0.415642,0.293631,0.189050,0.254749,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,0.435613,0.212751,0.417210,0.325902,0.371143,0.330430,0.398957,0.359061,0.288657,0.383312,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1416,0.283719,0.222758,0.302770,0.433613,0.342126,0.221158,0.342693,0.232092,0.133066,0.209742,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1417,0.124786,0.221111,0.444444,0.600520,0.263580,0.128000,0.266667,0.180000,0.300000,0.400000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1418,0.179931,0.225051,0.262166,0.317187,0.335347,0.177656,0.222070,0.133242,0.120288,0.224126,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.048321,0.0,0.0,0.0


In [9]:
data[(data.target<0.7)&(data.target>0.29)].target

Series([], Name: target, dtype: float64)

In [10]:
testdf.ix[data[(data.target<0.7)&(data.target>0.29)].index]

,writing_id,body
